In [1]:
from utils import *
from models import *
from loss import *
import numpy as np
import os
import matplotlib.pyplot as plt
from matplotlib import colors
from datetime import datetime

2025-04-27 22:14:39.011286: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
train_path = '../next_day_fires/next_day_wildfire_spread_train*'
val_path = '../next_day_fires/next_day_wildfire_spread_eval*'
test_path = '../next_day_fires/next_day_wildfire_spread_test*'
features = ['elevation', 'th', 'vs',  'tmmn', 'tmmx', 'sph', 'pr', 'pdsi', 'NDVI', 'population', 'erc', 'PrevFireMask', 'FireMask']

# Load Data

## Training Data

In [3]:
train_dataset = get_dataset(
      train_path,
      data_size=64,
      sample_size=32,
      batch_size=100,
      num_in_channels=12,
      compression_type=None,
      clip_and_normalize=False,
      clip_and_rescale=False,
      random_crop=True,
      center_crop=False)

train_inputs, train_labels = next(iter(train_dataset))

print(train_inputs.shape)
print(train_labels.shape)

torch.Size([100, 12, 32, 32])
torch.Size([100, 1, 32, 32])


In [4]:
titles = [
  'Elevation',
  'Wind\ndirection',
  'Wind\nvelocity',
  'Min\ntemp',
  'Max\ntemp',
  'Humidity',
  'Precip',
  'Drought',
  'Vegetation',
  'Population\ndensity',
  'Energy\nrelease\ncomponent',
  'Previous\nfire\nmask',
  'Fire\nmask'
]

n_rows = 5
n_features = train_inputs.shape[3] 
CMAP = colors.ListedColormap(['black', 'silver', 'orangered'])
BOUNDS = [-1, -0.1, 0.001, 1]
NORM = colors.BoundaryNorm(BOUNDS, CMAP.N)
keys = ['elevation', 'th', 'vs',  'tmmn', 'tmmx', 'sph', 'pr', 'pdsi', 'NDVI', 'population', 'erc', 'PrevFireMask']

In [5]:
# fig = plt.figure(figsize=(20,8))

# for i in range(n_rows):
#   for j in range(n_features + 1):
#     plt.subplot(n_rows, n_features + 1, i * (n_features + 1) + j + 1)
#     if i == 0:
#       plt.title(titles[j], fontsize=15)
#     if j < n_features - 1:
#       plt.imshow(train_inputs[i, j, :, :], cmap='viridis')
#     if j == n_features - 1:
#       plt.imshow(train_inputs[i, -1, :, :], cmap=CMAP, norm=NORM)
#     if j == n_features:
#       plt.imshow(train_labels[i, 0, :, :], cmap=CMAP, norm=NORM) 
#     plt.axis('off')
# plt.tight_layout()

## Vlidation Data

In [6]:
val_dataset = get_dataset(
      val_path,
      data_size=64,
      sample_size=32,
      batch_size=100,
      num_in_channels=12,
      compression_type=None,
      clip_and_normalize=False,
      clip_and_rescale=False,
      random_crop=True,
      center_crop=False)

val_inputs, val_labels = next(iter(val_dataset))

print(val_inputs.shape)
print(val_labels.shape)

torch.Size([100, 12, 32, 32])
torch.Size([100, 1, 32, 32])


## Test Data

In [7]:
test_dataset = get_dataset(
      test_path,
      data_size=64,
      sample_size=32,
      batch_size=100,
      num_in_channels=12,
      compression_type=None,
      clip_and_normalize=False,
      clip_and_rescale=False,
      random_crop=True,
      center_crop=False)

test_inputs, test_labels = next(iter(val_dataset))

print(test_inputs.shape)
print(test_labels.shape)

torch.Size([100, 12, 32, 32])
torch.Size([100, 1, 32, 32])


In [8]:
# Build DataLoader
train_loader = DataLoader(train_dataset, batch_size=None, num_workers=0, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=None, num_workers=0, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=None, num_workers=0, pin_memory=True)

# Model Initialization

In [9]:
class UNet(nn.Module):
    def __init__(self, kernel_size, stride, padding):
        super().__init__()

        # First we need to make the Encoder
        # There are 5 Encoder modules, each consisting of two convolution encoder layers to 
        # increase the number of channels and a maxpooling layer to downsample
        self.e11 = nn.Conv2d(12, 32, kernel_size=kernel_size, stride=stride, padding=padding)
        self.e12 = nn.Conv2d(32, 32, kernel_size=kernel_size, stride=stride, padding=padding)
        self.maxpool1 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.e21 = nn.Conv2d(32, 64, kernel_size=kernel_size, stride=stride, padding=padding)
        self.e22 = nn.Conv2d(64, 64, kernel_size=kernel_size, stride=stride, padding=padding)
        self.maxpool2 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.e31 = nn.Conv2d(64, 128, kernel_size=kernel_size, stride=stride, padding=padding)
        self.e32 = nn.Conv2d(128, 128, kernel_size=kernel_size, stride=stride, padding=padding)
        self.maxpool3 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.e41 = nn.Conv2d(128, 256, kernel_size=kernel_size, stride=stride, padding=padding)
        self.e42 = nn.Conv2d(256, 256, kernel_size=kernel_size, stride=stride, padding=padding)
        self.maxpool4 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.e51 = nn.Conv2d(256, 512, kernel_size=kernel_size, stride=stride, padding=padding)
        self.e52 = nn.Conv2d(512, 512, kernel_size=kernel_size, stride=stride, padding=padding)


        # Next, we will make the Decoder 
        # There will be 4 decoder modules, each upsampling using transposed convolution
        # followed by two convolution decoder layers to recover the original image shape
        self.upconv1 = nn.ConvTranspose2d(512, 256, kernel_size=2, stride=2)
        self.d11 = nn.Conv2d(512, 256, kernel_size=kernel_size, stride=stride, padding=padding)
        self.d12 = nn.Conv2d(256, 256, kernel_size=kernel_size, stride=stride, padding=padding)

        self.upconv2 = nn.ConvTranspose2d(256, 128, kernel_size=2, stride=2)
        self.d21 = nn.Conv2d(256, 128, kernel_size=kernel_size, stride=stride, padding=padding)
        self.d22 = nn.Conv2d(128, 128, kernel_size=kernel_size, stride=stride, padding=padding)

        self.upconv3 = nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2)
        self.d31 = nn.Conv2d(128, 64, kernel_size=kernel_size, stride=stride, padding=padding)
        self.d32 = nn.Conv2d(64, 64, kernel_size=kernel_size, stride=stride, padding=padding)

        self.upconv4 = nn.ConvTranspose2d(64, 32, kernel_size=2, stride=2)
        self.d41 = nn.Conv2d(64, 32, kernel_size=kernel_size, stride=stride, padding=padding)
        self.d42 = nn.Conv2d(32, 32, kernel_size=kernel_size, stride=stride, padding=padding)


        # Finally, we use a final output layer to create a single channel prediction
        self.output = nn.Conv2d(32, 1, kernel_size=1)


    # Now we create a forward pass pipeline for input processing through the model
    def forward(self, x):
        # Each encoder layer uses ReLU activation
        e11 = relu(self.e11(x))
        e12 = relu(self.e12(e11))
        maxpool1 = self.maxpool1(e12)

        e21 = relu(self.e21(maxpool1))
        e22 = relu(self.e22(e21))
        maxpool2 = self.maxpool2(e22)

        e31 = relu(self.e31(maxpool2))
        e32 = relu(self.e32(e31))
        maxpool3 = self.maxpool3(e32)

        e41 = relu(self.e41(maxpool3))
        e42 = relu(self.e42(e41))
        maxpool4 = self.maxpool4(e42)

        e51 = relu(self.e51(maxpool4))
        e52 = relu(self.e52(e51))

        # At each decoder level, we must concatenate corresponding encoder level outputs to retain image information
        upconv1 = self.upconv1(e52)
        upconv1 = torch.cat([upconv1, e42], dim=1)
        d11 = self.d11(upconv1)
        d12 = self.d12(d11)

        upconv2 = self.upconv2(e42)
        upconv2 = torch.cat([upconv2, e32], dim=1)
        d21 = self.d21(upconv2)
        d22 = self.d22(d21)

        upconv3 = self.upconv3(d22)
        upconv3 = torch.cat([upconv3, e22], dim=1)
        d31 = self.d31(upconv3)
        d32 = self.d32(d31)

        upconv4 = self.upconv4(d32)
        upconv4 = torch.cat([upconv4, e12], dim=1)
        d41 = self.d41(upconv4)
        d42 = self.d42(d41)

        # Use sigmoid activation on the output layers
        out = self.output(d42)
        out = sigmoid(out)

        return out

### Hyperparameters

In [10]:
# parameter setup
folder_path = './UNET_models'

kernel_size = 3
stride = 1
padding = 1

num_epochs = 200
patience = 50

bce_weight = 0.5 # Coefficient for Binary Cross Entropy loss in loss function

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### Loss Function

In [11]:
# Loss function setup
# Create a class for Dice Loss
class DiceLoss(nn.Module):
    # Initializing the loss function will require a smoothing parameter to prevent division by zero 
    def __init__(self, smooth=1e-6):
        super(DiceLoss, self).__init__()
        self.smooth = smooth
    
    # The loss function is defined in a forward pass as 2*(intersection)/(pred + true)
    def forward(self, pred, true):
        pred = pred.view(-1)
        true = true.view(-1)
        intersection = (pred*true).sum()
        dice = (2*intersection + self.smooth) / (pred.sum() + true.sum() + self.smooth)
        return 1 - dice

# Create a class for BCE Loss
class BCEDiceLoss(nn.Module):
    # Initializing the loss function will require a BCE weight parameter to determine the weigth of each loss 
    def __init__(self, bce_weight=0.5):
        super().__init__()
        # Declare both loss functions
        self.bce = nn.BCELoss()
        self.dice = DiceLoss()
        self.bce_weight = bce_weight

    # The loss function is defined by taking the weighted sum of BCE Loss and Dice Loss
    def forward(self, pred, true):
        bce_loss = self.bce(pred, true)
        dice_loss = self.dice(pred, true)
        return bce_weight * bce_loss + (1-bce_weight) * dice_loss

### Model Initialization

In [12]:
# Create an instance of UNet
model = UNet(kernel_size, stride, padding)
model.to(device)

UNet(
  (e11): Conv2d(12, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (e12): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (maxpool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (e21): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (e22): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (maxpool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (e31): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (e32): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (maxpool3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (e41): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (e42): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (maxpool4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (e51): Conv2d(256, 512, kerne

# Model Training

In [ ]:
# Train UNET
best_val_loss = float('inf')
epochs_no_improve = 0
best_model_state = None
best_model_filename = None

# Define the optimizer and loss function
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
criterion = BCEDiceLoss(bce_weight=0.5)

# Train over the entire dataset every epoch
for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    # Train in batches to improve efficiency
    for batch in train_loader:
        optimizer.zero_grad(set_to_none=True)
        images, true_mask = batch
        images = images.to(device)
        true_mask = true_mask.to(device)

        # Make predictions on the training images
        pred_mask = model(images)

        # Calculate the loss and take an optimization step based on the predictions
        loss = 0
        loss += criterion(pred_mask, true_mask)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    # Average the loss over all batches
    avg_loss = total_loss / len(train_loader)

    # Now, validate the model
    model.eval()
    val_loss = 0

    # Gradients are not used as validation is used to find the best model without further optimization
    with torch.no_grad():
        # Validate in batches
        for batch in val_loader:
            images, true_mask = batch
            images = images.to(device)
            true_mask = true_mask.to(device)
            
            # Make predictions on the validation images
            pred_mask = model(images)

            # Calculate the loss
            loss = 0
            loss += criterion(pred_mask, true_mask)
            val_loss += loss.item()
    
    # Average the validation loss over all batches
    avg_val_loss = val_loss / len(val_loader)
    
    print(f"Epoch {epoch + 1}/{num_epochs} - Train Loss: {avg_loss:.4f} - Validation Loss: {avg_val_loss:.4f}")

    # Best model saving & early stopping
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        epochs_no_improve = 0
        best_model_state = model.state_dict() 
        best_model_filename = f"{folder_path}/best_model_{str(best_val_loss)}_{datetime.now().strftime('%Y%m%d_%H%M%S')}.pth"
        print("Validation loss improved. Best model updated.")
    else:
        epochs_no_improve += 1
        print(f"No improvement for {epochs_no_improve} epoch(s).")
    
    if epochs_no_improve >= patience:
        print("Early stopping triggered!")
        break

        

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Epoch 1/200 - Train Loss: 2.4593 - Validation Loss: 1.2341
Validation loss improved. Best model updated.
Epoch 2/200 - Train Loss: 0.7500 - Validation Loss: 0.5606
Validation loss improved. Best model updated.
Epoch 3/200 - Train Loss: 0.5532 - Validation Loss: 0.5509
Validation loss improved. Best model updated.
Epoch 4/200 - Train Loss: 0.5444 - Validation Loss: 0.5510
No improvement for 1 epoch(s).


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/torch/utils/data/dataloader.py:643: UserWarning: Length of IterableDataset <utils.TFRecordPyTorchDataset object at 0x13a25f410> was reported to be 124 (when accessing len(dataloader)), but 125 samples have been fetched. 
  warnings.warn(warn_msg)


Epoch 5/200 - Train Loss: 0.5463 - Validation Loss: 0.5587
No improvement for 2 epoch(s).
Epoch 6/200 - Train Loss: 0.5382 - Validation Loss: 0.5526
No improvement for 3 epoch(s).
Epoch 7/200 - Train Loss: 0.5391 - Validation Loss: 0.5502
Validation loss improved. Best model updated.
Epoch 8/200 - Train Loss: 0.5332 - Validation Loss: 0.5451
Validation loss improved. Best model updated.
Epoch 9/200 - Train Loss: 0.5281 - Validation Loss: 0.5418
Validation loss improved. Best model updated.
Epoch 10/200 - Train Loss: 0.5300 - Validation Loss: 0.5554
No improvement for 1 epoch(s).
Epoch 11/200 - Train Loss: 0.5275 - Validation Loss: 0.5511
No improvement for 2 epoch(s).
Epoch 12/200 - Train Loss: 0.5235 - Validation Loss: 0.5406
Validation loss improved. Best model updated.
Epoch 13/200 - Train Loss: 0.5249 - Validation Loss: 0.5435
No improvement for 1 epoch(s).
Epoch 14/200 - Train Loss: 0.5172 - Validation Loss: 0.5479
No improvement for 2 epoch(s).
Epoch 15/200 - Train Loss: 0.5146 -

In [13]:
# save the best model
if best_model_state is not None:
    torch.save(best_model_state, best_model_filename)
    print(f"Best model saved to {best_model_filename}")

Best model saved to ./gen_models/best_model_27.718893954628392_20250426_202626.pth


# Model Testing

In [14]:
# test UNET
model.load_state_dict(torch.load(best_model_filename, map_location=device))
model.eval()

# Create arrays to store all predictions and all ground truth masks
pred_mask_all, true_mask_all = [], []

with torch.no_grad():
    for batch in testloader:
        images, true_mask = batch
        images = images.to(device)
        true_mask = true_mask.to(device)

        # Make test predictions
        pred_mask = model(images)

        pred_mask_all.append(pred_mask.cpu().numpy())
        true_mask_all.append(true_mask.cpu().numpy())

    # Concatenate all batches into a single array for performance                        
    preds = np.concatenate(pred_mask_all, axis=0)   
    trues = np.concatenate(true_mask_all, axis=0)        

    # Squeeze from 4 dimensions into 3 since we use only one channel
    if preds.ndim == 4 and preds.shape[1] == 1:
        preds = preds.squeeze(1)
        trues = trues.squeeze(1)

    # Binarize predictions
    preds  = (preds > 0.5).astype(np.uint8)
    trues  = trues.astype(np.uint8)

    preds = preds.flatten()
    trues = trues.flatten()

    # Calculate TP, FP, FN for evaluation metrics
    tp = np.logical_and(preds==1, trues==1).sum()
    fp = np.logical_and(preds==1, trues==0).sum()
    fn = np.logical_and(preds==0, trues==1).sum()

    # Calculate evaluation metrics
    smooth = 1e-6
    dice = (2*tp + smooth) / (2*tp + fp + fn + smooth)
    iou  = (tp + smooth)   / (tp + fp + fn + smooth)
    recall = tp / (tp + fn + smooth)

                             
    # Dice coefficient
    # IoU (Jaccard Index)
    # Recall - We care a lot about false negatives
    metrics = {
        'Dice Coefficient': dice,
        'IoU': iou,
        'Recall': recall
        }
    
    print("\n **Test Results:**")
    for key, value in metrics.items():
        print(f"{key}: {value:.4f}")


 **Test Results:**
Dice Coefficient: 0.0867
IoU: 0.0453
Recall: 0.0782
